# Distributed Demo Script

This part serves as a demo for you to try the algorithm, I will keep updating this part whenever something woth powerful effect was found.

### Firstly, import all packages you need.

In [1]:
import tensorflow as tf
from tensorflow.keras import layers
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedShuffleSplit
import pandas as pd
import numpy as np
np.random.seed(0)
from keras.utils import np_utils

Using TensorFlow backend.


### Secondly, load the train and test data, also, encode the label.

If you do not understand the Chinese Comment, just ignore them(They are same as the English ones).

In [2]:
train = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')
def encode(train, test):
    # Encoding the species of leaves by LabelEncoder
    # 用LabelEncoder为叶子的种类标签编码，labels对象是训练集上的标签列表
    label_encoder = LabelEncoder().fit(train.species)
    labels = label_encoder.transform(train.species)
    classes = list(label_encoder.classes_)
    # Deleting unnecessary variables in both training and test data.
    # 此处把不必要的训练集和测试集的列删除
    train = train.drop(['species', 'id'], axis=1)
    test = test.drop('id', axis=1)
    return train, labels, test, classes
train, labels, test, classes = encode(train, test)

### Data Processing

In [3]:
# Normalization
# 这里只是标准化训练集的特征值
scaler = StandardScaler().fit(train.values)
scaled_train = scaler.transform(train.values)
# Split with splitRate=0.1
# 把数据集拆分成训练集和测试集，测试集占10%
sss = StratifiedShuffleSplit(test_size=0.1, random_state=23)
for train_index, valid_index in sss.split(scaled_train, labels):
    X_train, X_valid = scaled_train[train_index], scaled_train[valid_index]
    y_train, y_valid = labels[train_index], labels[valid_index]

In [4]:
# 每个输入通道的大小是64位，一共3个通道
nb_features = 64 
nb_class = len(classes)
# Reshape ==> (Sample size,features,channel)
#  把输入数据集reshape成keras喜欢的格式：（样本数，通道大小，通道数）
X_train_r = np.zeros((len(X_train), nb_features, 3))
# 这里的做法是先把所有元素初始化成0之后，再把刚才的数据集中的数据赋值过来
X_train_r[:, :, 0] = X_train[:, :nb_features]
X_train_r[:, :, 1] = X_train[:, nb_features:128]
X_train_r[:, :, 2] = X_train[:, 128:]
# reshape valid data
# 验证集也要reshape一下
X_valid_r = np.zeros((len(X_valid), nb_features, 3))
X_valid_r[:, :, 0] = X_valid[:, :nb_features]
X_valid_r[:, :, 1] = X_valid[:, nb_features:128]
X_valid_r[:, :, 2] = X_valid[:, 128:]

In [5]:
# One-hot encoder
y_train = np_utils.to_categorical(y_train, nb_class)
y_valid = np_utils.to_categorical(y_valid, nb_class)

### Building Model

In [6]:
model = tf.keras.Sequential()
# 一维卷积层用了512个卷积核，输入是64*3的格式
# 此处要注意，一维卷积指的是卷积核是1维的，而不是卷积的输入是1维的，1维指的是卷积方式
model.add(layers.Conv1D(input_shape=(64, 3), filters=512, kernel_size=1))
model.add(layers.Activation('relu'))
model.add(layers.Flatten())
model.add(layers.Dropout(0.4))
model.add(layers.Dense(2048, activation='relu'))
model.add(layers.Dense(1024, activation='relu'))
model.add(layers.Dense(nb_class))
model.add(layers.Activation('softmax'))

optimizer = tf.train.GradientDescentOptimizer(0.2)

model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 64, 512)           2048      
_________________________________________________________________
activation (Activation)      (None, 64, 512)           0         
_________________________________________________________________
flatten (Flatten)            (None, 32768)             0         
_________________________________________________________________
dropout (Dropout)            (None, 32768)             0         
_________________________________________________________________
dense (Dense)                (None, 2048)              67110912  
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              2098176   
_________________________________________________________________
dense_2 (Dense)              (None, 99)                101475    
__________

### Distributed settings(strategy)

In [7]:
strategy = tf.contrib.distribute.MirroredStrategy()
config = tf.estimator.RunConfig(train_distribute=strategy)

INFO:tensorflow:Initializing RunConfig with distribution strategies.
INFO:tensorflow:Not using Distribute Coordinator.


### Transform keras model into an estimator for distributed training

In [8]:
keras_estimator = tf.keras.estimator.model_to_estimator(
      keras_model=model,
      config=config,
      model_dir='dir/')

INFO:tensorflow:Using the Keras model provided.
INFO:tensorflow:Using config: {'_model_dir': 'dir/', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': <tensorflow.contrib.distribute.python.mirrored_strategy.MirroredStrategy object at 0x1c21cd4d68>, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1c21cd4e80>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1, '_distribute_coordinator_mode': None}


### Build feed dictionary for model

In [9]:
def input_fn(X,Y,epochs, batch_size):
    x = X
    y = Y
    x = tf.cast(x, tf.float32)
    dataset = tf.data.Dataset.from_tensor_slices((x, y))
    SHUFFLE_SIZE = 5000
    dataset = dataset.shuffle(SHUFFLE_SIZE).repeat(epochs).batch(batch_size)
    dataset = dataset.prefetch(2)
    return dataset

### Training

In [10]:
BATCH_SIZE = 512
EPOCHS = 15

keras_estimator.train(lambda:input_fn(
                            X = X_train_r,
                            Y = y_train,
                            epochs=EPOCHS,
                            batch_size=BATCH_SIZE)
                     )

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Warm-starting with WarmStartSettings: WarmStartSettings(ckpt_to_initialize_from='dir/keras/keras_model.ckpt', vars_to_warm_start='.*', var_name_to_vocab_info={}, var_name_to_prev_var_name={})
INFO:tensorflow:Warm-starting from: ('dir/keras/keras_model.ckpt',)
INFO:tensorflow:Warm-starting variable: conv1d/kernel; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: conv1d/bias; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: dense/kernel; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: dense/bias; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: dense_1/kernel; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: dense_1/bias; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: dense_2/kernel; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: dense_2/bias; prev_var_name: Unchanged
INFO:tensorflow:C

### Testing

In [11]:
keras_estimator.evaluate(lambda:input_fn(
                                  X = X_valid_r,
                                  Y = y_valid,
                                  epochs=1,
                                  batch_size=BATCH_SIZE))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-11-05-17:26:10
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from dir/model.ckpt-27
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-11-05-17:26:11
INFO:tensorflow:Saving dict for global step 27: accuracy = 0.9898989, global_step = 27, loss = 0.05634372
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 27: dir/model.ckpt-27


{'accuracy': 0.9898989, 'loss': 0.05634372, 'global_step': 27}